In [31]:
!pip install -q -U google-genai
!pip install pydantic
!pip install seqeval
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 18.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 19.3 MB/s eta 0:00:00 0:00:01


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("eriktks/conll2003")

# Access the train, validation, and test splits
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

# Print a sample
print(train_data[0])

/Users/francescostella/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [3]:
# Print a sample from the training data
example = train_data[11001]
print("Example from the training data:")
print(example)
print("Tokens:", example["tokens"])
print("Labels:", example["ner_tags"])

Example from the training data:
{'id': '11001', 'tokens': ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.'], 'pos_tags': [11, 8, 15, 22, 22, 15, 22, 6, 22, 6, 22, 21, 10, 21, 15, 12, 16, 16, 22, 22, 21, 30, 38, 35, 22, 22, 22, 7], 'chunk_tags': [11, 12, 13, 11, 12, 13, 11, 0, 11, 0, 11, 12, 0, 11, 13, 11, 12, 12, 12, 12, 12, 3, 21, 13, 11, 12, 12, 0], 'ner_tags': [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]}
Tokens: ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.']
Labels: [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]


In [4]:
print("Training set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))

Training set size: 14041
Validation set size: 3250
Test set size: 3453


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [ ]:
import csv
import os


def parse_response(tokens : list, response_labels : list, true_labels : list) -> list: 
    '''
    Store the response in a list of lists where the first element is the token, the second element 
    is the predicted label and the third is the true label
    '''
    response_labels = response_labels.split(":")
    response_labels = response_labels[1].strip('\n').split(',')
    if (len(response_labels) != len(tokens)):
        if (len(response_labels) > len(tokens)):
            response_labels = response_labels[:len(tokens)]
        if (len(response_labels) < len(tokens)):
            response_labels = response_labels + ['0'] * (len(tokens) - len(response_labels))

    temp = []
    for i in range(len(tokens)):
        pred_label = int(response_labels[i].strip())
        
        assert (pred_label >= 0 and pred_label <= 8), "Predicted label is out of range"
        temp.append([tokens[i], pred_label, true_labels[i]])

    return temp

def save_to_csv(tokens : list, pred_labels : list, true_labels : list, filename : str) -> None:
    file_exists = os.path.isfile(filename)
    # Write header only if the file didn't exist before
    if not file_exists:
        with open(filename, 'a', newline='') as csvfile:
            header = ['token', 'pred', 'true']
            writer = csv.writer(csvfile)
            writer.writerow(header)
    data = [[tokens[i], pred_labels[i], true_labels[i]] for i in range(len(tokens)) if pred_labels[i] != 0 and true_labels[i] != 0]
    # Remove duplicates
    # Open the file in append mode and write data to analysis purpose
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [19]:
# Take predicted labels and for each token save the label in a list to be used for voting
def store_predicted_labels(pred_labels : list, votes : list) -> None:
    for i in range(len(pred_labels)):
        votes[i].append(pred_labels[i])

In [24]:
import random

# Sample 300 random elements from the test set
sampled_test_data = random.sample(list(test_data), 300)

# Print the first few samples to verify
for i, sample in enumerate(sampled_test_data[:5]):  # Display the first 5 samples
    print(f"Sample {i + 1}:")
    print(sample)
    print("\n")

Sample 1:
{'id': '1645', 'tokens': ['12/15/2007', '850M', '4.85', '4.85'], 'pos_tags': [11, 11, 11, 11], 'chunk_tags': [11, 12, 12, 12], 'ner_tags': [0, 0, 0, 0]}


Sample 2:
{'id': '986', 'tokens': ['Tambang', 'Timah', 'at', '$', '15.575', 'in', 'London', '.'], 'pos_tags': [39, 22, 15, 3, 11, 15, 22, 7], 'chunk_tags': [21, 11, 13, 11, 12, 13, 11, 0], 'ner_tags': [3, 4, 0, 0, 0, 0, 5, 0]}


Sample 3:
{'id': '529', 'tokens': ['SAN', 'FRANCISCO', '10', '3', '0', '325', '198'], 'pos_tags': [22, 22, 11, 11, 11, 11, 11], 'chunk_tags': [11, 12, 12, 12, 12, 12, 12], 'ner_tags': [3, 4, 0, 0, 0, 0, 0]}


Sample 4:
{'id': '302', 'tokens': ['The', 'West', 'Indies', 'vice-captain', 'struggled', 'for', 'timing', 'during', 'his', '36-minute', 'stay', 'at', 'the', 'crease', 'before', 'chipping', 'a', 'ball', 'from', 'medium', 'pacer', 'Tom', 'Moody', 'straight', 'to', 'Shane', 'Warne', 'at', 'mid-wicket', '.'], 'pos_tags': [12, 22, 22, 16, 38, 15, 21, 15, 29, 16, 21, 15, 12, 21, 15, 39, 12, 21, 15, 2

In [ ]:
#from pydantic import BaseModel
from google import genai

for j in range(len(sampled_test_data)):
    # tokens = train_data[j]['tokens']
    # true_labels = train_data[j]['ner_tags']
    tokens = sampled_test_data[j]['tokens']
    true_labels = sampled_test_data[j]['ner_tags']
    votes = [[] for _ in range(len(tokens))]
    model = "gemma-3-27b-it"
    #for i in range(5):
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    # Send request to Gemma
    response = client.models.generate_content(
        model=model,
        contents=f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
        The sentence is: '{tokens}'
        This sentence contains exactly {len(tokens)} tokens.

        Print only the number associated with the NER tag for each of the {legn(tokens)} tokens, using the tag-to-number mapping provided above.
        Your answer MUST follow the format: ner_tags: 0, 1, 2, 0, 0, 0
        The number of output NER tags MUST be exactly {len(tokens)}, one for each token in the order they appear in the sentence.
        Do not include any other text or explanations.
        """
    )

    # Parse the response
    data = parse_response(tokens, response.text, true_labels)
        # # Store predicted labels for voting
        # store_predicted_labels([data[i][1] for i in range(len(data))], votes)
    # Extract for each token the most voted label
    #votes = [max(set(vote), key=vote.count) for vote in votes]
    #save_to_csv(tokens, votes, true_labels, "data/data_test.csv")
    save_to_csv(tokens, [item[1] for item in data], true_labels, "data/vanilla_test_300.csv")

In [133]:
import csv
import os

# def save_basic_decomposed_qa_to_csv(tokens: list, pred_labels: dict, true_labels: dict, filename: str) -> None:
#     """
#     Save the token, predicted labels, and true labels to a CSV file for Basic Decomposed-QA.

#     Args:
#         tokens (list): List of tokens in the sentence.
#         pred_labels (dict): Dictionary of predicted labels in the format {'PER': 'entity1, entity2', ...}.
#         true_labels (dict): Dictionary of true labels in the format {'PER': 'entity1, entity2', ...}.
#         filename (str): Path to the CSV file (relative to the 'data' folder).
#     """
#     # Ensure the 'data' directory exists
#     os.makedirs("data", exist_ok=True)
#     print(pred_labels)
#     print(pred_labels['B-PER'])
    
#     # Full path to the file in the 'data' directory
#     filepath = os.path.join("data", filename)
    
#     file_exists = os.path.isfile(filepath)
    
#     # Write header only if the file didn't exist before
#     if not file_exists:
#         with open(filepath, 'a', newline='') as csvfile:
#             header = ['token', 'pred', 'true']
#             writer = csv.writer(csvfile)
#             writer.writerow(header)
    
#     # Prepare data for each token
#     data = []
#     for token in tokens:
#         predicted_label = None
#         true_label = None
        
#         # Find the predicted label for the token
#         for label, entities in pred_labels.items():
#             if token in entities.split(', '):
#                 predicted_label = label
#                 break
        
#         # Find the true label for the token
#         for label, entities in true_labels.items():
#             if token in entities.split(', '):
#                 true_label = label
#                 break
        
#         # Append the token, predicted label, and true label
#         data.append([token, predicted_label if predicted_label else 'O', true_label if true_label else 'O'])
    
#     # Write data to the CSV file
#     with open(filepath, 'a', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows(data)
label_mapping = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

def save_basic_decomposed_qa_to_csv(tokens: list, pred_labels: dict, true_labels: dict, filename: str) -> None:
    """
    Save the token, predicted labels, and true labels to a CSV file for Basic Decomposed-QA.

    Args:
        tokens (list): List of tokens in the sentence.
        pred_labels (dict): Dictionary of predicted labels in the format {'PER': ['entity1', 'entity2'], ...}.
        true_labels (dict): Dictionary of true labels in the format {'PER': ['entity1', 'entity2'], ...}.
        filename (str): Path to the CSV file (relative to the 'data' folder).
    """
    # Ensure the 'data' directory exists
    os.makedirs("data", exist_ok=True)
    filepath = os.path.join("data", filename)

    file_exists = os.path.isfile(filepath)

    if not file_exists:
        with open(filepath, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['token', 'pred', 'true'])

    # Create reverse lookup from token to label index
    def get_label_index(token_idx, label_dict):
        for label_str, entities in label_dict.items():
            if entities == 'None':
                continue
            tokens_in_label = [t.strip() for t in entities.split(',')]
            if tokens[token_idx] in tokens_in_label:
                # Convert label to index
                for idx, tag in label_mapping.items():
                    if tag == label_str:
                        return idx
        return 0  # 'O'

    # Write data
    with open(filepath, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i, token in enumerate(tokens):
            pred_idx = get_label_index(i, pred_labels)
            true_idx = get_label_index(i, true_labels)
            writer.writerow([token, pred_idx, true_idx])

In [119]:
from google import genai

# Mapping of label numbers to readable tags
label_mapping = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

# List of NER labels and their descriptions (without distinction between B- and I-)
ner_labels = {
    'PER': 'Person',
    'ORG': 'Organization',
    'LOC': 'Location',
    'MISC': 'Miscellaneous'
}

# Iterate over the first 5 examples of the dataset
for j in range(len(test_data)):
    tokens = sampled_test_data[j]['tokens']
    true_labels = sampled_test_data[j]['ner_tags']
    votes = [[] for _ in range(len(tokens))]  # Initialize votes for majority voting

    # Repeat the process 5 times for majority voting
    #for iteration in range(5):
    # Re-initialize the client for each sentence
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

    # Build a dictionary for the true labels
    true_label_dict = {label: [] for label in ner_labels.keys()}
    for token, true_label in zip(tokens, true_labels):
        readable_label = label_mapping[true_label]
        if readable_label != 'O':  # Ignore tokens without labels
            main_label = readable_label.split('-')[-1]  # Get PER, ORG, LOC, MISC
            true_label_dict[main_label].append(token)

    # Convert lists to comma-separated strings
    for key in true_label_dict:
        true_label_dict[key] = ', '.join(true_label_dict[key]) if true_label_dict[key] else 'None'

    context = f"""Given the following NER tags: {{'Person': 'PER', 'Organization': 'ORG', 'Location': 'LOC', 'Miscellaneaous': 'MISC'}}, determine the Named Entity Recognition (NER) tags for the following sentence.
    The sentence is: '{tokens}'."""
    results = []  # To save the model's responses
    
    # Iterate over each NER label
    for label, description in ner_labels.items():
        # Build the question for the current label
        question = f"Question: What are the named entities labeled as '{description}' in the text? You only have to answer with the entity name, if there are multiple entities then separate them with a comma. If there are no entities, answer with 'None'. Finally, remember that a token can only be classified once.\n"

        # Send the request to the model
        response = client.models.generate_content(
            model="gemma-3-27b-it",
            contents=f"{context}\n{question}\n"
        )

        # Save the response and update the context
        answer = response.text.strip()
        results.append({label: answer})
        context += f"\n{question}\n{answer}\n"

    # Parse the response and update votes
    for token_idx, token in enumerate(tokens):
        predicted_labels = []
        for result in results:
            label = list(result.keys())[0]  # Get the label (e.g., 'ORG')
            predicted_entities = result[label].split(', ') if result[label] != 'None' else []
            if token in predicted_entities:
                predicted_labels.append(label)
        # Add the predicted label for this token to the votes
        votes[token_idx].append(predicted_labels[0] if predicted_labels else 'O')

    # Perform majority voting for each token
    final_labels = [max(set(vote), key=vote.count) for vote in votes]

    # Build the final predicted labels in the desired format
    final_label_dict = {label: [] for label in ner_labels.keys()}
    for token, final_label in zip(tokens, final_labels):
        if final_label != 'O':  # Ignore tokens without labels
            final_label_dict[final_label].append(token)

    # Convert lists to comma-separated strings
    for key in final_label_dict:
        final_label_dict[key] = ', '.join(final_label_dict[key]) if final_label_dict[key] else 'None'

    # Print the results for the current sample
    print(f"Results for sample {j + 1}:")
    print(f"Tokens: {tokens}")
    print("Predicted Labels:")
    print(final_label_dict)
    print("True Labels:")
    print(true_label_dict)
    print("\n")

    # Save the results to a CSV file
    save_basic_decomposed_qa_to_csv(tokens, final_label_dict, true_label_dict, "basic_decomposed_qa_results.csv")

KeyboardInterrupt: 

In [107]:
import csv
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# Step 1: Define label mapping (inverse of your prompt's dictionary)
index_to_label = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-ORG',
    4: 'I-ORG',
    5: 'B-LOC',
    6: 'I-LOC',
    7: 'B-MISC',
    8: 'I-MISC'
}

# Step 2: Read the CSV and convert predictions and true labels
true_seqs = []
pred_seqs = []
current_true = []
current_pred = []

with open('data/vanilla_test_300.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        token = row['token']
        true_label = index_to_label[int(row['true'])]
        pred_label = index_to_label[int(row['pred'])]

        current_true.append(true_label)
        current_pred.append(pred_label)

        # Optional: if your dataset contains sentence boundaries, split sequences here
        # For now we assume it's one sentence
    true_seqs.append(current_true)
    pred_seqs.append(current_pred)

# Step 3: Compute metrics
print("Precision:", precision_score(true_seqs, pred_seqs))
print("Recall:", recall_score(true_seqs, pred_seqs))
print("F1 Score:", f1_score(true_seqs, pred_seqs))

# Optional detailed report
print("\nDetailed classification report:\n")
print(classification_report(true_seqs, pred_seqs))


Precision: 0.5513626834381551
Recall: 0.6368038740920097
F1 Score: 0.5910112359550561

Detailed classification report:

              precision    recall  f1-score   support

         LOC       0.56      0.86      0.68       139
        MISC       0.43      0.29      0.35        41
         ORG       0.26      0.23      0.24       118
         PER       0.79      0.90      0.84       115

   micro avg       0.55      0.64      0.59       413
   macro avg       0.51      0.57      0.53       413
weighted avg       0.53      0.64      0.57       413



In [ ]:
#Decomposed-QA
#from google import genai
# import google.generativeai as genai
# import time


# label_groups = [(1,2), (3,4), (5,6), (7,8)]
# #labels_class = {'Person': 'PER', 'Organization': 'ORG', 'Location': 'LOC', 'Miscellaneaous': 'MISC'}

# for j in range(len(sampled_test_data)):
# #for j in range(6):
#     tokens = sampled_test_data[j]['tokens']
#     true_labels = sampled_test_data[j]['ner_tags']
#     results = []
#     # Configure with your API key
#     genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
#     #genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

#     model = genai.GenerativeModel('gemma-3-27b-it')
#     #model = genai.GenerativeModel('gemini-2.0-flash')
#     chat = model.start_chat() # No initial history
#     intro_message = f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
#     The sentence is: '{tokens}'.
#     This sentence contains exactly {len(tokens)} tokens.
#     Remember that a token can only be classified once.
#     Which are the tokens labeled as ’{label_mapping[group[0]]}’ and '{label_mapping[group[1]]} in the sentence? You only have to answer with entity name. If there are multiple tokens for a single category then separate the list of tokens with commas. If there are no entities, answer with 'None'"""
#     try:
#         chat.send_message(intro_message)
#     except Exception as e:
#         time.sleep(30)
#         chat.send_message(intro_message)
#     for group in label_groups[1:]:
#         #print(label_mapping[group[0]], label_mapping[group[1]])
#         try:
#             response = chat.send_message(f"""Which are the tokens labeled as ’{label_mapping[group[0]]}’ and '{label_mapping[group[1]]}' in the text? You only have to answer with entity name and each entity must correspond to a token. If there are multiple tokens for a single category then separate the list of tokens with commas. If there are no entities, answer with 'None'""")
#         except Exception as e:
#             time.sleep(30)
#             response = chat.send_message(f"""Which are the tokens labeled as ’{label_mapping[group[0]]}’ and '{label_mapping[group[1]]}' in the text? You only have to answer with entity name and each entity must correspond to a token. If there are multiple tokens for a single category then separate the list of tokens with commas. If there are no entities, answer with 'None'""")
#         #print(response.text)
#         answer = response.text.strip()
#         results.append({group[1] : answer})
#     print("Results", results)
    
#     # Transform results into the pred_labels dictionary
#     pred_labels = {label: [] for label in label_mapping.values()}
#     for result in results:
#         label = list(result.keys())[0]  
#         entities = result[label].split(', ') if result[label] != 'None' else []
#         pred_labels[label] = ', '.join(entities) if entities else 'None'

#     # Transform true labels into the required dictionary format
#     true_label_dict = {label: [] for label in label_mapping.values()}
#     for token, true_label in zip(tokens, true_labels):
#         readable_label = label_mapping[true_label]
#         if readable_label != 'O':  
#             # main_label = readable_label.split('-')[-1]
#             # print(main_label)
#             # true_label_dict[main_label].append(token)
#             true_label_dict[readable_label].append(token)


#     # Convert lists to comma-separated strings
#     for key in true_label_dict:
#         true_label_dict[key] = ', '.join(true_label_dict[key]) if true_label_dict[key] else 'None'

#     #print(pred_labels)
#     #print(true_label_dict)
#     # Save the results to a CSV file
#     save_basic_decomposed_qa_to_csv(tokens, pred_labels, true_label_dict, "qa_results_bio.csv")






import google.generativeai as genai
import os
import time

category_to_index = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

# label_mapping = {
#     0: 'O',
#     1: 'B-PER',
#     2: 'I-PER',
#     3: 'B-ORG',
#     4: 'I-ORG',
#     5: 'B-LOC',
#     6: 'I-LOC',
#     7: 'B-MISC',
#     8: 'I-MISC'
# }

label_groups = [(1, 2), (3, 4), (5, 6), (7, 8)]

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
#model = genai.GenerativeModel('gemma-3-27b-it')  # or 'gemini-2.0-flash'
model = genai.GenerativeModel('gemini-2.5-flash-preview-04-17')  # or 'gemini-2.0-flash'

#for j in range(len(sampled_test_data)):
for j in range(50):
    tokens = sampled_test_data[j]['tokens']
    true_labels = sampled_test_data[j]['ner_tags']
    results = []
    
    chat = model.start_chat()

    # Send general instructions once
    intro_message = (
        f"""Given the following NER tags: {{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}}, determine the Named Entity Recognition (NER) tags for the following sentence.
        If there are multiple tokens for a single category then separate the list of tokens with commas.
        The response should be for example: 'B-PER: entity1, entity2, entity3', 'I-PER: entity4, entity5'.
        If there are no entities, answer with 'None', no further explanation is needed.
        The sentence is: '{tokens}'.
        This sentence contains exactly {len(tokens)} tokens.
        Remember that a token can only be classified once.
        If there are multiple tokens for a single category then separate the list of tokens with commas.
        If there are no entities, just answer with 'None'."""
    )

    try:
        chat.send_message(intro_message)
        time.sleep(5)
    except Exception as e:
        print("Intro message failed, retrying in 30s:", e)
        time.sleep(30)
        chat.send_message(intro_message)

    # Delay between requests to avoid rate limit
    for group in label_groups:
        label_1 = label_mapping[group[0]]
        label_2 = label_mapping[group[1]]
        print(label_1, label_2)
        prompt = (
            f"""Which are the tokens labeled as '{label_1}' and '{label_2}' in the text? """
            f"""You only have to answer with category : entity name and each entity must correspond to a token. """
            
        )

        try:
            response = chat.send_message(prompt)
            time.sleep(5)
        except Exception as e:
            print(f"Request failed for {label_1}/{label_2}, retrying in 30s:", e)
            time.sleep(30)
            response = chat.send_message(prompt)
        print('Response: ', response.text)
        #time.sleep(2)  # Small delay to reduce request rate
        answer = response.text.strip()
        results.append({label_mapping[group[1]]: answer})  # Use label string directly

    # Build predicted labels as token lists
    pred_labels = {label: [] for label in label_mapping.values()}
    for result in results:
        label = list(result.keys())[0]
        entities = result[label].split(', ') if result[label] != 'None' else []
        pred_labels[label] = entities

    # Build true label dictionary
    true_label_dict = {label: [] for label in label_mapping.values()}
    for token, true_label in zip(tokens, true_labels):
        label_str = label_mapping[true_label]
        if label_str != 'O':
            true_label_dict[label_str].append(token)

    # Convert to strings for storage
    for key in true_label_dict:
        true_label_dict[key] = ', '.join(true_label_dict[key]) if true_label_dict[key] else 'None'
    for key in pred_labels:
        pred_labels[key] = ', '.join(pred_labels[key]) if pred_labels[key] else 'None'

    save_basic_decomposed_qa_to_csv(tokens, pred_labels, true_label_dict, "qa_results_bio_flash_25.csv")


B-PER I-PER
Response:  None
B-ORG I-ORG
Response:  None
B-LOC I-LOC
Response:  None
B-MISC I-MISC
Response:  None
B-PER I-PER
Response:  None
B-ORG I-ORG
Response:  B-ORG : Tambang, I-ORG : Timah
B-LOC I-LOC
Response:  B-LOC : London
B-MISC I-MISC
Response:  None
B-PER I-PER
Response:  None
B-ORG I-ORG
Response:  None
B-LOC I-LOC
Response:  B-LOC : SAN
I-LOC : FRANCISCO
B-MISC I-MISC
Response:  None
B-PER I-PER
Response:  B-PER : Tom, B-PER : Shane, I-PER : Moody, I-PER : Warne
B-ORG I-ORG
Response:  B-ORG : West, I-ORG : Indies
B-LOC I-LOC
Response:  None
B-MISC I-MISC
Response:  None


KeyboardInterrupt: 

In [106]:
import csv
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Load true and predicted labels
true_labels = []
pred_labels = []

with open('data/qa_results.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        true_labels.append(row['true'])
        pred_labels.append(row['pred'])

# Compute token-level metrics
print("Precision:", precision_score(true_labels, pred_labels, average='macro', zero_division=0))
print("Recall:", recall_score(true_labels, pred_labels, average='macro', zero_division=0))
print("F1 Score:", f1_score(true_labels, pred_labels, average='macro', zero_division=0))

# Detailed report
print("\nClassification Report:\n")
print(classification_report(true_labels, pred_labels, zero_division=0))


Precision: 0.7281962411375407
Recall: 0.8122881731878271
F1 Score: 0.7430039993271146

Classification Report:

              precision    recall  f1-score   support

         LOC       0.74      0.87      0.80        39
        MISC       0.11      0.50      0.18        10
           O       0.99      0.92      0.96       578
         ORG       0.83      0.77      0.80        39
         PER       0.97      1.00      0.98        60

    accuracy                           0.91       726
   macro avg       0.73      0.81      0.74       726
weighted avg       0.96      0.91      0.93       726

